In [5]:
import numpy as np
input_data=np.load("gradient_emo.npy")
label=np.load("label_landmark.npy")
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(input_data, label, test_size=0.3, 
                     stratify=label,
                     random_state=0)

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

In [3]:
%load_ext watermark
%watermark -u -d -v -p numpy,tensorflow,matplotlib

Last updated: 2021-06-11

Python implementation: CPython
Python version       : 3.7.10
IPython version      : 7.24.1

numpy     : 1.20.2
tensorflow: 2.4.1
matplotlib: 3.4.2



In [4]:
import tensorflow as tf

## 그래프를 생성합니다
g = tf.Graph()
with g.as_default():
    x = tf.compat.v1.placeholder(dtype=tf.float32,
                       shape=(None), name='x')
    w = tf.Variable(2.0, name='weight')
    b = tf.Variable(0.7, name='bias')

    z = w*x + b
    init = tf.compat.v1.global_variables_initializer()

## 세션을 만들고 그래프 g를 전달합니다
with tf.compat.v1.Session(graph=g) as sess:
    ## w와 b를 초기화합니다.
    sess.run(init)
    ## z를 평가합니다.
    for t in [1.0, 0.6, -1.8]:
        print('x=%4.1f --> z=%4.1f'%(
              t, sess.run(z, feed_dict={x:t})))

x= 1.0 --> z= 2.7
x= 0.6 --> z= 1.9
x=-1.8 --> z=-2.9


In [7]:

print('행: %d,  열: %d' %(X_train.shape[0], 
                                 X_train.shape[1]))

print('행: %d,  열: %d' %(X_test.shape[0], 
                                 X_test.shape[1]))

## 평균을 0으로 만들고 표준 편차로 나눕니다.
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val
 
del X_train, X_test
 
print(X_train_centered.shape, y_train.shape)

print(X_test_centered.shape, y_test.shape)

행: 24869,  열: 59
행: 10659,  열: 59
(24869, 59) (24869,)
(10659, 59) (10659,)


/home/lab/anaconda3/envs/tensorflow/lib/python3.7/site-packages/numpy/core/_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [8]:
np.random.seed(123)

In [9]:
y_train_onehot = tf.keras.utils.to_categorical(y_train)
 
print('처음 3개 레이블: ', y_train[:3])
print('\n처음 3개 레이블 (원-핫):\n', y_train_onehot[:3])

처음 3개 레이블:  [3 2 4]

처음 3개 레이블 (원-핫):
 [[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]


In [18]:
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Dense(
        units=200,    
        input_dim=X_train_centered.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    tf.keras.layers.Dense(
        units=200,    
        input_dim=500,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    tf.keras.layers.Dense(
        units=y_train_onehot.shape[1],    
        input_dim=500,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 200)               12000     
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 1407      
Total params: 53,607
Trainable params: 53,607
Non-trainable params: 0
_________________________________________________________________


In [20]:
sgd_optimizer = tf.keras.optimizers.SGD(
    lr=0.001, decay=1e-7, momentum=.9)

model.compile(optimizer=sgd_optimizer,
              loss='categorical_crossentropy')

In [21]:
history = model.fit(X_train_centered, y_train_onehot,
                    batch_size=64, epochs=50,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
350/350 [==============================] - 1s 1ms/step - loss: 1.8872 - val_loss: 1.8242
Epoch 2/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8185 - val_loss: 1.8161
Epoch 3/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8124 - val_loss: 1.8141
Epoch 4/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8117 - val_loss: 1.8135
Epoch 5/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8105 - val_loss: 1.8129
Epoch 6/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8108 - val_loss: 1.8132
Epoch 7/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8057 - val_loss: 1.8136
Epoch 8/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8086 - val_loss: 1.8132
Epoch 9/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8115 - val_loss: 1.8128
Epoch 10/50
350/350 [==============================] - 0s 1ms/step - loss: 1.8122 - val_loss: 1.8133

In [22]:
y_train_pred = model.predict_classes(X_train_centered, 
                                     verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0) 
train_acc = correct_preds / y_train.shape[0]

print('처음 3개 예측: ', y_train_pred[:3])
print('훈련 정확도: %.2f%%' % (train_acc * 100))

처음 3개 예측:  [3 3 3]
훈련 정확도: 25.17%


In [23]:
y_test_pred = model.predict_classes(X_test_centered, 
                                    verbose=0)

correct_preds = np.sum(y_test == y_test_pred, axis=0) 
test_acc = correct_preds / y_test.shape[0]
print('테스트 정확도: %.2f%%' % (test_acc * 100))

테스트 정확도: 25.17%


In [24]:
import numpy as np

X = np.array([1, 1.4, 2.5]) ## 첫 번째 원소는 1이어야 합니다
w = np.array([0.4, 0.3, 0.5])

def net_input(X, w):
    return np.dot(X, w)

def logistic(z):
    return 1.0 / (1.0 + np.exp(-z))

def logistic_activation(X, w):
    z = net_input(X, w)
    return logistic(z)

print('P(y=1|x) = %.3f' % logistic_activation(X, w))

P(y=1|x) = 0.888
